In [35]:
import pandas as pd
import numpy as np
import random

# file_names = ['jpm', 'jp morgan', 'jpmorgan']
file_names = ['jpm']

data = pd.DataFrame()
for fName in file_names:
    data = data.append(pd.read_csv("reddit-data/" + fName + '.csv', sep=",", encoding="utf-8", error_bad_lines = False, header = 0))

In [2]:
data.shape

(30550, 6)

In [3]:
target_date = "2015-01-01 00:00:00"
temp = data[data['created'] > target_date]

In [4]:
subreddit_omit = ['formula', 'F1', "nascar", "AmericanHorrorStory"]

for sub in subreddit_omit:
    temp = temp[~temp['subreddit'].str.lower().str.contains(sub, na = False)]
    
temp

,author,body,created,id,score,subreddit
0,Ganamaa,"Can any of us buy JPM? If not, why is this here?",2019-06-26 09:50:39,es24ppb,1,CryptoCurrency
1,legitcreed444,With a bold statement like that and the confid...,2019-06-26 08:00:32,es1vyyd,1,stocks
2,investorH,Fuck..rip my MU rips..should've inversed WSB.....,2019-06-26 07:30:40,es1tlut,1,wallstreetbets
5,Celestin2512,You forget that the JPM coin is intended for i...,2019-06-25 23:01:25,es0gprb,1,AllThingsCrypto
6,wittybob,This coin is backed by a US dollar. What makes...,2019-06-25 22:54:33,es0g3nx,1,AllThingsCrypto
7,Shockinga,"Bankers will benefit from JPM coins, although...",2019-06-25 22:48:27,es0fk0u,1,AllThingsCrypto
8,atdharris,I wouldn't buy it. All bank stocks are trash a...,2019-06-25 22:18:59,es0d0l7,1,stocks
10,AristosTotalis,Yeah of course. Tho I was talking to a GS/JPM/...,2019-06-25 21:39:01,es09rx6,1,cscareerquestions
11,GnomeFetish,New to Market profile? [Check out this guide](...,2019-06-25 13:06:30,erzkks9,1,thewallstreet
12,sc_shay20,Why JPM?,2019-06-25 11:53:16,erzg4q5,1,stocks


In [44]:
import nltk
from nltk.corpus import stopwords
import re #regular expression
import string

#HappyEmoticons
emoticons_happy = set([
    ':-)', ':)', ';)', ':o)', ':]', ':3', ':c)', ':>', '=]', '8)', '=)', ':}',
    ':^)', ':-D', ':D', '8-D', '8D', 'x-D', 'xD', 'X-D', 'XD', '=-D', '=D',
    '=-3', '=3', ':-))', ":'-)", ":')", ':*', ':^*', '>:P', ':-P', ':P', 'X-P',
    'x-p', 'xp', 'XP', ':-p', ':p', '=p', ':-b', ':b', '>:)', '>;)', '>:-)',
    '<3'
    ])

# Sad Emoticons
emoticons_sad = set([
    ':L', ':-/', '>:/', ':S', '>:[', ':@', ':-(', ':[', ':-||', '=L', ':<',
    ':-[', ':-<', '=\\', '=/', '>:(', ':(', '>.<', ":'-(", ":'(", ':\\', ':-c',
    ':c', ':{', '>:\\', ';('
    ])

#Emoji patterns
emoji_pattern = re.compile("["
         u"\U0001F600-\U0001F64F"  # emoticons
         u"\U0001F300-\U0001F5FF"  # symbols & pictographs
         u"\U0001F680-\U0001F6FF"  # transport & map symbols
         u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
         u"\U00002702-\U000027B0"
         u"\U000024C2-\U0001F251"
         "]+", flags=re.UNICODE)

emoticons = emoticons_happy.union(emoticons_sad)

# Clean tweets from tweepy
def clean_tweets(tweet):
 
    stop_words = set(stopwords.words('english'))
    word_tokens = nltk.word_tokenize(tweet)
    #after tweepy preprocessing the colon symbol left remain after      #removing mentions
    tweet = re.sub(r':', '', tweet)
    tweet = re.sub(r'‚Ä¶', '', tweet)
    #replace consecutive non-ASCII characters with a space
    tweet = re.sub(r'[^\x00-\x7F]+',' ', tweet)
    #remove emojis from tweet
    tweet = emoji_pattern.sub(r'', tweet)
    #filter using NLTK library append it to a string
    filtered_tweet = [w for w in word_tokens if not w in stop_words]
    filtered_tweet = []
    #looping through conditions
    for w in word_tokens:
    #check tokens against stop words , emoticons and punctuations
        if w not in stop_words and w not in emoticons and w not in string.punctuation:
            filtered_tweet.append(w)
    return ' '.join(filtered_tweet)
    #print(word_tokens)
    #print(filtered_sentence)return tweet

In [ ]:
#placeholder method => using Random number
def getSentiment(body):
#     body = clean_tweets(body)
    sent_list = [0,2,4]
    return (random).choice(sent_list)

for row in temp.itertuples():
    temp.loc[row.Index, 'created'] = row.created.split(" ", 1)[0]
    temp.loc[row.Index, 'sentiment'] = getSentiment(row.body)

temp.head()

In [31]:
df = temp.groupby(['created','sentiment']).size()
# result = df['sentiment'].agg([np.mean, np.size])\
#     .rename(columns = {"mean":"average sentiment", "size":"count"})
    
df = df.unstack(level=-1).fillna(0)
# id_df = grouped['ID'].apply(lambda x: pd.Series(x.values)).unstack()
df

sentiment,0.0,2.0,4.0
created,,,
2015-01-01,1.0,1.0,0.0
2015-01-03,1.0,0.0,1.0
2015-01-04,0.0,1.0,2.0
2015-01-05,3.0,4.0,1.0
2015-01-06,1.0,3.0,2.0
2015-01-07,3.0,4.0,4.0
2015-01-08,0.0,0.0,2.0
2015-01-09,1.0,1.0,1.0
2015-01-10,2.0,0.0,0.0


In [34]:
df.to_csv("output/clean_reddit_sentiment_count.csv")